# Матричные разложения [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) (50 баллов)

Будем сжимать [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) для [задачи сегментации людей](https://www.kaggle.com/datasets/laurentmih/aisegmentcom-matting-human-datasets).

## Задания
Задание 1. Напишите реализацию SVD layer (35 Баллов)

Задание 2. Для Нескольких слоев proj в Decoder Head (5 Баллов):
    -  Произвидите замену
    -  Заморозте градиенты замененого слоя
    -  Сделате fine-tuning процедуру
    -  Разморозьте градиенты. Потренеруйте опять. Насколько улучшилось качество?
    -  Какие при этом получились accuracy и iou?
Задание 3. Попробуйте сжать все слои сразу (5 баллов)




Задание 4. Попробуйте разные ранги. Покажите результрующие график или таблицу в accuracy и iou (5 баллов). В свободной форме. Если с перебором рангов не задалось или у вас и так получися хороший результат, и вы можете обосновать выбор ранга/степени сжатия/l cингулярных чисел  - опишите какая схема обучения (layer-wise, single shot etc.) оказалось эффективнее. Какой подход вы бы предпочтли в продакшене? В целом не будет каких-то penalty за final accuracy/iou - в целом хотелось бы прочитать мысли уважаемых студентов.

https://jacobgil.github.io/deeplearning/tensor-decompositions-deep-learning

## Скачаем вспомогательный код и чекпоинт бейзлайна (модели-учителя)

In [ ]:
!wget -O hw_files_3.zip 'https://www.dropbox.com/scl/fi/9h0jmaxspgm755uqjmwev/hw_files_3.zip?rlkey=kytgclnvuixzoa6gys7wufpj1&dl=0'
!unzip -o hw_files_3.zip

### Скачаем датасет

Датасет находится по ссылке https://drive.google.com/file/d/1YOEDzZvhLb2DS1Yn7p7MSs41ou3ZBXUq/view?usp=sharing

Нужно его скачать и распаковать в папке, в которой находится ноутбук либо скачать его на гугл диск, и далее подключиться к гугл диску с помношью команд ниже

### Установим библиотеки

In [ ]:
# %%capture
# !pip install torch transformers datasets tensorboard pillow
# !pip install tensorly
# !pip install evaluate

In [2]:
import sys
from pathlib import Path
import os

# Calculate the absolute path to the parent directory (two levels up)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))

# Convert the Path object to a string and add it to sys.path
sys.path.append(str(parent_dir))

import typing as tp
import torch
import numpy as np
from copy import deepcopy
from datasets import load_metric
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm.auto import tqdm

from torch import nn
from transformers.models.segformer.modeling_segformer import SegformerLayer

from utils.data import init_dataloaders
from utils.model import evaluate_model
from utils.model import init_model_with_pretrain, init_model_student_with_pretrain

from utils.train import TrainParams

/home/gvasserm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# маппинг названия классов и индексов
id2label = {
    0: "background",
    1: "human",
}
label2id = {v: k for k, v in id2label.items()}

mse_loss = nn.MSELoss()
kl_loss = nn.KLDivLoss()

save_dir = 'assignment3/runs/svd'

tb_writer = SummaryWriter(save_dir)

Создадим лоадеры:

In [4]:
train_dataloader, valid_dataloader = init_dataloaders(
    root_dir="/home/gvasserm/data/matting_human_dataset",
    batch_size=8,
    num_workers=2,
)

/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [5]:
len(train_dataloader),len(valid_dataloader)

(3443, 63)

# SVD layer

Давайте напишем SVD layer и функции замены слоев (задание 1)

In [6]:
def truncated_svd(W, l, transpose=False):
    """Compress the weight matrix W of an inner product (fully connected) layer
    using truncated SVD.
    Parameters:
    W: N x M weights matrix
    l: number of singular values to retain
    Returns:
    Ul, L: matrices such that W \approx Ul*L
    """
    # посчитаем SVD
    U, s, V = torch.linalg.svd(W, full_matrices=False)  # Compute the SVD
    #

    Ul = U[:, :l]
    sl = s[:l]  # Keep the first l singular values
    V = V.t()
    Vl = V[:l, :]
    
    # Обьеденим  Sigma_l and V_l
    SV = torch.diag(sl) @ Vl
    
    if transpose: # Транспонируем
        Ul, SV = Ul.T, SV.T
        
    return Ul, SV


class TruncatedSVDLayer(nn.Module):
    def __init__(self, replaced_gemm, rank=None, preserve_ratio=0.9, device='cpu', transpose=False):
        super().__init__()
        self.replaced_gemm = replaced_gemm
        self.W = self.replaced_gemm.weight
        self.b = self.replaced_gemm.bias
        self.transpose = transpose

        print("W = {}".format(self.W.shape))
        if rank is None:
            rank = int(preserve_ratio * self.W.size(0))
        # считаем U and SV
        self.U, self.SV = truncated_svd(self.W.data, rank,transpose = self.transpose)
        print("U = {}".format(self.U.shape))
        # Cоздаем слой иницализорованный U - нужного размера
        self.fc_u = nn.Linear(self.U.size(1), self.U.size(0)).to(device)
        self.fc_u.weight.data = self.U

        print("SV = {}".format(self.SV.shape))
        # Cоздаем слой иницализорованный SV - нужного размера
        self.fc_sv = nn.Linear(self.SV.size(1), self.SV.size(0), bias=False).to(device)
        self.fc_sv.weight.data = self.SV
        # забываем старый слой
        #self.W = ...
        #self.replaced_gemm = ...

        del self.W, self.replaced_gemm

    def forward(self, x):
        # применим SV к x
        x = self.fc_sv(x)
        # применим U
        x = self.fc_u(x)
        # не забудим про bias
        if self.b is not None:
            x += self.b.view(1, -1).expand_as(x)
        return x

def create_small_network(
    model,
    decode_head_layer_id=-1,
    preserve_ratio=0.1,
    device='cuda',
    transpose=False
):
    """Выбрали слой,  сжали его и прозвели замену"""
    proj = model.decode_head.linear_c[decode_head_layer_id].proj
    compressed = TruncatedSVDLayer(proj, 
                                   rank=None, 
                                   preserve_ratio=preserve_ratio,
                                   device=device, 
                                   transpose=transpose)
    model.decode_head.linear_c[decode_head_layer_id].proj = compressed
    return model

In [13]:
def get_n_params(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params

def disable_old_layers_grads(model, decode_head_layer_id = -1):
    model.decode_head.linear_c[decode_head_layer_id].requires_grad = False

def enable_all_grads(model):
    for name,param in model.named_parameters():
        param.requires_grad = True

Создадим модель и загрузим в нее чекпоинт с прошлых домашних работ:
- и проверим как влияет замена одного SVD слоя

In [8]:
model = init_model_with_pretrain(label2id=label2id, id2label=id2label, pretrain_path='runs/ckpt_4.pth')

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
full_metrics = evaluate_model(model, valid_dataloader, id2label)

/home/gvasserm/dev/ml_acceleration/utils/model.py:59: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("mean_iou")
/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.9325449922186004
Mean accuracy: 0.964695608705032


In [10]:
decomposed_model = create_small_network(deepcopy(model))

W = torch.Size([64, 64])
U = torch.Size([64, 6])
SV = torch.Size([6, 64])


In [11]:
decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)

/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.5012150963185376
Mean accuracy: 0.7018947674318701


In [14]:
# дававйте проверим iou drop и как сократилось число параметров
(full_metrics['mean_iou']-decomposed_metrics['mean_iou'],
get_n_params(decomposed_model)/get_n_params(model))

(0.4313298959000629, 0.9783292833525873)

In [15]:
decomposed_model = deepcopy(model) # возьмем не сжатую модель
for i in [-1,-2,-3,-4]: #
    # cжимаем все слои сразу
    decomposed_model = create_small_network(deepcopy(decomposed_model),decode_head_layer_id=i,transpose=False)

W = torch.Size([64, 64])
U = torch.Size([64, 6])
SV = torch.Size([6, 64])
W = torch.Size([64, 40])
U = torch.Size([64, 6])
SV = torch.Size([6, 40])
W = torch.Size([64, 16])
U = torch.Size([64, 6])
SV = torch.Size([6, 16])
W = torch.Size([64, 8])
U = torch.Size([64, 6])
SV = torch.Size([6, 8])


In [16]:
decomposed_model.decode_head

SegformerDecodeHead(
  (linear_c): ModuleList(
    (0): SegformerMLP(
      (proj): TruncatedSVDLayer(
        (fc_u): Linear(in_features=6, out_features=64, bias=True)
        (fc_sv): Linear(in_features=8, out_features=6, bias=False)
      )
    )
    (1): SegformerMLP(
      (proj): TruncatedSVDLayer(
        (fc_u): Linear(in_features=6, out_features=64, bias=True)
        (fc_sv): Linear(in_features=16, out_features=6, bias=False)
      )
    )
    (2): SegformerMLP(
      (proj): TruncatedSVDLayer(
        (fc_u): Linear(in_features=6, out_features=64, bias=True)
        (fc_sv): Linear(in_features=40, out_features=6, bias=False)
      )
    )
    (3): SegformerMLP(
      (proj): TruncatedSVDLayer(
        (fc_u): Linear(in_features=6, out_features=64, bias=True)
        (fc_sv): Linear(in_features=64, out_features=6, bias=False)
      )
    )
  )
  (linear_fuse): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.

In [17]:
decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)

/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.24388964158632342
Mean accuracy: 0.5054494255792118


In [18]:
# дававйте проверим iou drop и как сократилось число параметров
(full_metrics['mean_iou']-decomposed_metrics['mean_iou'],
get_n_params(decomposed_model)/get_n_params(model))

(0.688655350632277, 0.9626073908828958)

## Train Loop

In [19]:
from dataclasses import dataclass
from datasets import load_metric

@dataclass
class TrainParams:
    n_epochs: int
    lr: float
    batch_size: int
    n_workers: int
    device: torch.device

    loss_weight: float
    last_layer_loss_weight: float
    intermediate_attn_layers_weights: tp.Tuple[float, float, float, float]
    intermediate_feat_layers_weights: tp.Tuple[float, float, float, float]
    # возможно, в ваших экспериментах захотите добавить что-то ещё

In [21]:
train_params = TrainParams(
    n_epochs=1,
    lr=6e-5,
    batch_size=8,
    n_workers=2,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_weight=1,
    last_layer_loss_weight=0.,
    intermediate_attn_layers_weights=(0, 0, 0, 1.),
    intermediate_feat_layers_weights=(0, 0, 0, 1.),
)

In [22]:
student_teacher_attention_mapping = {0: 0, 1: 1, 2: 2, 3: 3}

mse_loss = nn.MSELoss()
kl_loss = nn.KLDivLoss()

def calc_last_layer_loss(student_logits, teacher_logits, weight):
    return mse_loss(student_logits, teacher_logits) * weight

def calc_intermediate_layers_attn_loss(student_attentions, teacher_attentions, weights, student_teacher_attention_mapping):
    intermediate_kl_loss = 0
    for i, (stud_attn_idx, teach_attn_idx) in enumerate(student_teacher_attention_mapping.items()):
        intermediate_kl_loss += weights[i] * kl_loss(
            input=torch.log(student_attentions[stud_attn_idx]),
            target=teacher_attentions[teach_attn_idx],
        )
    return intermediate_kl_loss

def calc_intermediate_layers_feat_loss(student_feats, teacher_feats, weights):
    intermediate_mse_loss = 0.
    for i in range(len(student_feats)):
        intermediate_mse_loss += weights[i] * mse_loss(
            input=student_feats[i],
            target=teacher_feats[i],
        )
    return intermediate_mse_loss

In [23]:
def train(
    teacher_model,
    student_model,
    train_params: TrainParams,
    student_teacher_attention_mapping,
):
    metric = load_metric('mean_iou')
    teacher_model.to(train_params.device)
    student_model.to(train_params.device)

    teacher_model.eval()

    train_dataloader, valid_dataloader = init_dataloaders(
        root_dir="/home/gvasserm/data/matting_human_dataset",
        batch_size=train_params.batch_size,
        num_workers=train_params.n_workers,
    )

    optimizer = torch.optim.AdamW(student_model.parameters(), lr=train_params.lr)
    step = 0
    for epoch in range(train_params.n_epochs):
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        for idx, batch in pbar:
            student_model.train()
            # get the inputs;
            pixel_values = batch['pixel_values'].to(train_params.device)
            labels = batch['labels'].to(train_params.device)

            optimizer.zero_grad()

            # forward + backward + optimize
            student_outputs = student_model(
                pixel_values=pixel_values,
                labels=labels,
                output_attentions=True,
                output_hidden_states=True,
            )
            loss, student_logits = student_outputs.loss, student_outputs.logits

            # Чего это мы no_grad() при тренировке поставили?!
            with torch.no_grad():
                teacher_output = teacher_model(
                    pixel_values=pixel_values,
                    labels=labels,
                    output_attentions=True,
                    output_hidden_states=True,
                )


            last_layer_loss = calc_last_layer_loss(
                student_logits,
                teacher_output.logits,
                train_params.last_layer_loss_weight,
            )

            student_attentions, teacher_attentions = student_outputs.attentions, teacher_output.attentions
            student_hidden_states, teacher_hidden_states = student_outputs.hidden_states, teacher_output.hidden_states

            intermediate_layer_att_loss = calc_intermediate_layers_attn_loss(
                student_attentions,
                teacher_attentions,
                train_params.intermediate_attn_layers_weights,
                student_teacher_attention_mapping,
            )

            intermediate_layer_feat_loss = calc_intermediate_layers_feat_loss(
                student_hidden_states,
                teacher_hidden_states,
                train_params.intermediate_feat_layers_weights,
            )

            total_loss = loss* train_params.loss_weight + last_layer_loss
            if intermediate_layer_att_loss is not None:
                total_loss += intermediate_layer_att_loss

            if intermediate_layer_feat_loss is not None:
                total_loss += intermediate_layer_feat_loss

            step += 1

            total_loss.backward()
            optimizer.step()
            pbar.set_description(f'total loss: {total_loss.item():.3f}')

            for loss_value, loss_name in (
                (loss, 'loss'),
                (total_loss, 'total_loss'),
                (last_layer_loss, 'last_layer_loss'),
                (intermediate_layer_att_loss, 'intermediate_layer_att_loss'),
                (intermediate_layer_feat_loss, 'intermediate_layer_feat_loss'),
            ):
                if loss_value is None: # для выключенной дистилляции атеншенов
                    continue
                tb_writer.add_scalar(
                    tag=loss_name,
                    scalar_value=loss_value.item(),
                    global_step=step,
                )

        #после модификаций модели обязательно сохраняйте ее целиком, чтобы подгрузить ее в случае чего
        torch.save(
            {
                'model': student_model,
                'state_dict': student_model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
            },
            f'{save_dir}/ckpt_{epoch}.pth',
        )

        eval_metrics = evaluate_model(student_model, valid_dataloader, id2label)

        for metric_key, metric_value in eval_metrics.items():
            if not isinstance(metric_value, float):
                continue
            tb_writer.add_scalar(
                tag=f'eval_{metric_key}',
                scalar_value=metric_value,
                global_step=epoch,
            )


In [24]:
train(
        teacher_model=model,
        student_model=decomposed_model,
        train_params=train_params,
        student_teacher_attention_mapping=student_teacher_attention_mapping
    )

/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

Mean_iou: 0.9268582353398809
Mean accuracy: 0.9614815151535334


# Layer-wise подход (Задание 2)
Реализуейте последовательное сжатие слоев из decoder head. Cмотрите детальное описание в заголовке ноубука.

In [25]:
# teacher from distill
model = init_model_with_pretrain(
    label2id=label2id,
    id2label=id2label, 
    pretrain_path='runs/ckpt_4.pth',
)


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
num_mlp_in_head = len(model.decode_head.linear_c) # посчитаем число proj слоев в голове-декодере

In [27]:
layer_ids = -(np.arange(num_mlp_in_head)+1) # возьмем их в обратном порядке - можете задать id в ручную обычным list

In [28]:
layer_ids

array([-1, -2, -3, -4])

In [30]:
#student from distill
decomposed_model = deepcopy(model) # возьмем не сжатую модель
iou_drops = [] # для сохранения результата =
compress_ratios = []

train_params = TrainParams(
    n_epochs=1,
    lr=18e-5,
    batch_size=24,
    n_workers=2,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_weight=1,
    last_layer_loss_weight=0.,
    intermediate_attn_layers_weights=(0, 0, 0, 1.),
    intermediate_feat_layers_weights=(0, 0, 0, 1.),)

student_teacher_attention_mapping = {0: 0, 1: 1, 2: 2, 3: 3}

train_dataloader, valid_dataloader = init_dataloaders(
    root_dir="/home/gvasserm/data/matting_human_dataset/",
    batch_size=24,
    num_workers=8)

full_metrics = evaluate_model(model, valid_dataloader, id2label)

for i in layer_ids: #
    
    # cледуем шагам из задания 2
    decomposed_model = create_small_network(deepcopy(decomposed_model), decode_head_layer_id=i,
        preserve_ratio=0.1,
        device='cuda',
        transpose=False)
    
    disable_old_layers_grads(decomposed_model, decode_head_layer_id = i)

    train(teacher_model=model,
        student_model=decomposed_model,
        train_params=train_params,
        student_teacher_attention_mapping=student_teacher_attention_mapping)

    enable_all_grads(decomposed_model)

    train(teacher_model=model,
        student_model=decomposed_model,
        train_params=train_params,
        student_teacher_attention_mapping=student_teacher_attention_mapping)
    
    compress_ratios.append(get_n_params(decomposed_model)/get_n_params(model))
    decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)
    diff = full_metrics['mean_iou']-decomposed_metrics['mean_iou']
    iou_drops.append(diff)

print(decomposed_model.decode_head.linear_c) # давайте напечатаем слои
decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label) # финальная аккураси

d = full_metrics['mean_iou']-decomposed_metrics['mean_iou']
# дававйте проверим iou drop и как сократилось число параметров
print(f'Metric Diff: {d}')
print(f'Model size compression ratio: {get_n_params(decomposed_model)/get_n_params(model)}')

print(iou_drops)
print(compress_ratios)

/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.9325450350995874
Mean accuracy: 0.9646956311875714
W = torch.Size([64, 64])
U = torch.Size([64, 6])
SV = torch.Size([6, 64])


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

Mean_iou: 0.9307831798001585
Mean accuracy: 0.9639562225767835


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

...

Mean_iou: 0.9324898470902958
Mean accuracy: 0.9651674861426278


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.9324898470902958
Mean accuracy: 0.9651674861426278
ModuleList(
  (0): SegformerMLP(
    (proj): TruncatedSVDLayer(
      (fc_u): Linear(in_features=6, out_features=64, bias=True)
      (fc_sv): Linear(in_features=8, out_features=6, bias=False)
    )
  )
  (1): SegformerMLP(
    (proj): TruncatedSVDLayer(
      (fc_u): Linear(in_features=6, out_features=64, bias=True)
      (fc_sv): Linear(in_features=16, out_features=6, bias=False)
    )
  )
  (2): SegformerMLP(
    (proj): TruncatedSVDLayer(
      (fc_u): Linear(in_features=6, out_features=64, bias=True)
      (fc_sv): Linear(in_features=40, out_features=6, bias=False)
    )
  )
  (3): SegformerMLP(
    (proj): TruncatedSVDLayer(
      (fc_u): Linear(in_features=6, out_features=64, bias=True)
      (fc_sv): Linear(in_features=64, out_features=6, bias=False)
    )
  )
)


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.9324898470902958
Mean accuracy: 0.9651674861426278
Metric Diff: 5.518800929160683e-05
Model size compression ratio: 0.9626073908828958
[0.001596953106323018, -0.00017213130681170696, -0.0013266744673938735, 5.518800929160683e-05]
[0.9783292833525873, 0.9659004899812771, 0.9627136198860694, 0.9626073908828958]


# Жмем все сразу (one shot) (задание 3)
Попробуйте сжать все слои одновременно и прозвести finetuning по аналогии с заданием 2. Вам просто нужно минимально изменить цикл из прошлого задания. При желании попробуй сравнить данный подход с заморозкой градиентов и без

In [31]:
num_mlp_in_head = len(model.decode_head.linear_c) # посчитаем число proj слоев в голове-декодере
layer_ids = -(np.arange(num_mlp_in_head)+1) # возьмем их в обратном порядке - можете задать id в ручную обычным list

print(f'Number proj in head: {layer_ids}')

#student from distill
decomposed_model = deepcopy(model) # возьмем не сжатую модель
iou_drops = [] # для сохранения результата =
compress_ratios = []

train_params = TrainParams(
    n_epochs=1,
    lr=18e-5,
    batch_size=24,
    n_workers=2,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_weight=1,
    last_layer_loss_weight=0.,
    intermediate_attn_layers_weights=(0, 0, 0, 1.),
    intermediate_feat_layers_weights=(0, 0, 0, 1.),)

student_teacher_attention_mapping = {0: 0, 1: 1, 2: 2, 3: 3}

train_dataloader, valid_dataloader = init_dataloaders(
    root_dir="/home/gvasserm/data/matting_human_dataset/",
    batch_size=24,
    num_workers=8)

full_metrics = evaluate_model(model, valid_dataloader, id2label)

decomposed_model = deepcopy(model) # возьмем не сжатую модель
for i in layer_ids: #
    # cжимаем все слои сразу
    decomposed_model = create_small_network(deepcopy(decomposed_model), decode_head_layer_id=i,
        preserve_ratio=0.1,
        device='cuda',
        transpose=False)
    
    disable_old_layers_grads(decomposed_model, decode_head_layer_id = i)


train(teacher_model=model,
    student_model=decomposed_model,
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping)


enable_all_grads(decomposed_model)

train(teacher_model=model,
    student_model=decomposed_model,
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping)


decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)
iou_diff_shot = full_metrics['mean_iou']-decomposed_metrics['mean_iou']

print(iou_diff_shot)
print(get_n_params(decomposed_model) / get_n_params(model))

Number proj in head: [-1 -2 -3 -4]


/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_cod

Mean_iou: 0.9325450350995874
Mean accuracy: 0.9646956311875714
W = torch.Size([64, 64])
U = torch.Size([64, 6])
SV = torch.Size([6, 64])
W = torch.Size([64, 40])
U = torch.Size([64, 6])
SV = torch.Size([6, 40])
W = torch.Size([64, 16])
U = torch.Size([64, 6])
SV = torch.Size([6, 16])
W = torch.Size([64, 8])
U = torch.Size([64, 6])
SV = torch.Size([6, 8])


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

Mean_iou: 0.9265246417234334
Mean accuracy: 0.9616980865896457


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

Mean_iou: 0.929185881701661
Mean accuracy: 0.9634119956713021


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.929185881701661
Mean accuracy: 0.9634119956713021
0.003359153397926362
0.9626073908828958


# Задание 4
Смотри описание в начале ноутбука. Просто хочется услышать/прочесть мысли Ваши мысли о рангах или и разных статегиях сжатия.

In [32]:
num_mlp_in_head = len(model.decode_head.linear_c) # посчитаем число proj слоев в голове-декодере
layer_ids = -(np.arange(num_mlp_in_head)+1) # возьмем их в обратном порядке - можете задать id в ручную обычным list

print(f'Number proj in head: {layer_ids}')

#student from distill
decomposed_model = deepcopy(model) # возьмем не сжатую модель
iou_drops = [] # для сохранения результата =
compress_ratios = []

train_params = TrainParams(
    n_epochs=1,
    lr=18e-5,
    batch_size=24,
    n_workers=2,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_weight=1,
    last_layer_loss_weight=0.,
    intermediate_attn_layers_weights=(0, 0, 0, 1.),
    intermediate_feat_layers_weights=(0, 0, 0, 1.),)

student_teacher_attention_mapping = {0: 0, 1: 1, 2: 2, 3: 3}

train_dataloader, valid_dataloader = init_dataloaders(
    root_dir="/home/gvasserm/data/matting_human_dataset/",
    batch_size=24,
    num_workers=8)

full_metrics = evaluate_model(model, valid_dataloader, id2label)

decomposed_model = deepcopy(model) # возьмем не сжатую модель
for i in layer_ids: #
    # cжимаем все слои сразу
    decomposed_model = create_small_network(deepcopy(decomposed_model), decode_head_layer_id=i,
        preserve_ratio=0.2,
        device='cuda',
        transpose=False)
    
    disable_old_layers_grads(decomposed_model, decode_head_layer_id = i)


train(teacher_model=model,
    student_model=decomposed_model,
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping)


enable_all_grads(decomposed_model)

train(teacher_model=model,
    student_model=decomposed_model,
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping)


decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)
iou_diff_shot = full_metrics['mean_iou']-decomposed_metrics['mean_iou']

print(iou_diff_shot)
print(get_n_params(decomposed_model) / get_n_params(model))

Number proj in head: [-1 -2 -3 -4]


/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_cod

Mean_iou: 0.9325450350995874
Mean accuracy: 0.9646956311875714
W = torch.Size([64, 64])
U = torch.Size([64, 12])
SV = torch.Size([12, 64])
W = torch.Size([64, 40])
U = torch.Size([64, 12])
SV = torch.Size([12, 40])
W = torch.Size([64, 16])
U = torch.Size([64, 12])
SV = torch.Size([12, 16])
W = torch.Size([64, 8])
U = torch.Size([64, 8])
SV = torch.Size([8, 8])


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

Mean_iou: 0.9278443852716403
Mean accuracy: 0.962560437935448


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/gvasserm/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is de

Mean_iou: 0.9309704711324012
Mean accuracy: 0.9636964269144526


/home/gvasserm/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for mean_iou contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/mean_iou/mean_iou.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean_iou: 0.9309704711324012
Mean accuracy: 0.9636964269144526
0.0015745639671862488
0.9759922452827683


## Отправка решения

Загрузите ноутбук на образовательную платформу.

In [ ]:
'''
PreserveRatio LayerByLayer ParamsRatio  mIOU   dIOU
1                  -           1      0.9327    0
0.1               Yes          0.96   0.9324    5e-5
0.1                No          0.96   0.9291    0.003
0.2                No          0.97   0.9309    0.001

'''

'''
Seems that the most accurate strategy is to do it layer by layer with freeze-unfreeze regime. However, the comparison is unfair since for layer-by-layer the ammount of fine tune epochs was 4 times bigger.

'''